Here is the system of differential equations: two equations for the velocity components $u,v$ and one equation for pressure:

$$\frac{\partial u}{\partial t}+u\frac{\partial u}{\partial x}+v\frac{\partial u}{\partial y} = -\frac{1}{\rho}\frac{\partial p}{\partial x}+\nu \left(\frac{\partial^2 u}{\partial x^2}+\frac{\partial^2 u}{\partial y^2} \right) $$


$$\frac{\partial v}{\partial t}+u\frac{\partial v}{\partial x}+v\frac{\partial v}{\partial y} = -\frac{1}{\rho}\frac{\partial p}{\partial y}+\nu\left(\frac{\partial^2 v}{\partial x^2}+\frac{\partial^2 v}{\partial y^2}\right) $$

$$\frac{\partial^2 p}{\partial x^2}+\frac{\partial^2 p}{\partial y^2} = -\rho\left(\frac{\partial u}{\partial x}\frac{\partial u}{\partial x}+2\frac{\partial u}{\partial y}\frac{\partial v}{\partial x}+\frac{\partial v}{\partial y}\frac{\partial v}{\partial y} \right)$$

$$\frac{\partial \gamma_{xx}}{\partial t} + u\frac{\partial \gamma_{xx}}{\partial x} +
v \frac{\partial \gamma_{xx}}{\partial y} = 2 \frac{\partial u}{\partial x}-2\omega\gamma_{xy}$$

$$\frac{\partial \gamma_{xy}}{\partial t} + u\frac{\partial \gamma_{xy}}{\partial x} +
v \frac{\partial \gamma_{xy}}{\partial y} = \frac{\partial u}{\partial y}+\frac{\partial v}{\partial x}+2\omega\gamma_{xx}$$

In [1]:
from fipy import *
#%matplotlib inline

In [2]:
L=1.0
ncells = 50
dL = L / ncells
dt = 1.e-4
U = 1
G = .2
viscosity = .1

In [3]:
mesh = Grid2D(nx = ncells, ny = ncells, dx = dL, dy = dL)
meshP = PeriodicGrid2DLeftRight(nx = ncells, ny = ncells, dx = dL, dy = dL)
pressure = CellVariable(mesh = mesh, name='pressure',value=0.)
xVelocity = CellVariable(mesh = meshP, name='X-velocity', hasOld=True)
yVelocity = CellVariable(mesh = meshP, name='Y-velocity', hasOld=True)
velocity = FaceVariable(mesh=meshP,rank=1)

X, Y = mesh.cellCenters
phase = CellVariable(mesh = mesh, name=r'$\phi$', hasOld=True)
#phase.constrain(1., (X-0.5)**2 + (Y-0.5)**2 < 5*dL)
dT = CellVariable(name = r'$\Delta T$', mesh = mesh, hasOld=True)
theta = ModularVariable(name = r'$\theta$',mesh = mesh, hasOld=True)
theta.constrain(-numerix.pi, mesh.exteriorFaces)
theta.value = -numerix.pi+0.0001

gamma_xx = CellVariable(mesh = mesh, name=r'$\gamma_{xx}$',value=0.,hasOld=True)
gamma_xy = CellVariable(mesh = mesh, name=r'$\gamma_{xy}$',value=0.,hasOld=True)

The initial condition is $u, v, p = 0$ everywhere, and the boundary conditions are:

$u=1$ at $y=1$ (the "lid");

$u, v=0$ on the other boundaries;

$\frac{\partial p}{\partial y}=0$ at $y=0$;

$p=0$ at $y=1$

$\frac{\partial p}{\partial x}=0$ at $x=0,1$



In [4]:
#xVelocity.constrain(0.,mesh.facesRight | mesh.facesLeft | mesh.facesBottom)
#xVelocity.constrain(U, mesh.facesTop)
#yVelocity.constrain(0., mesh.exteriorFaces)
#pressure.constrain(0.,mesh.facesTop)
#pressure.faceGrad.dot([0,1]).constrain(0.,mesh.facesBottom)
#pressure.faceGrad.dot([1,0]).constrain(0.,mesh.facesLeft)
#pressure.faceGrad.dot([1,0]).constrain(0.,mesh.facesRight)

The initial condition is $u, v, p=0$ everywhere, and at the boundary conditions are:

$u, v, p$ are periodic on $x=0,2$

$u, v =0$ at $y =0,2$

$\frac{\partial p}{\partial y}=0$ at $y =0,2$

$F=1$ everywhere.





In [5]:
xVelocity.constrain(0, mesh.facesTop | mesh.facesBottom)
yVelocity.constrain(0, mesh.facesTop | mesh.facesBottom)
pressure.faceGrad.dot([0,1]).constrain(0.,mesh.facesBottom)
pressure.faceGrad.dot([0,1]).constrain(0.,mesh.facesTop)
pressure.constrain(0.0,mesh.facesRight)
pressure.constrain(1.0,mesh.facesLeft)

We change previous equations by adding dependence on the solid phase. The solid phase is given by $\phi$ field. The equations are now:

$$\frac{\partial u}{\partial t}+u\frac{\partial u}{\partial x}+v\frac{\partial u}{\partial y} = -\frac{1}{\rho}\frac{\partial p}{\partial x}+\nu(1-P(\phi)) \left(\frac{\partial^2 u}{\partial x^2}+\frac{\partial^2 u}{\partial y^2} \right) +GP(\phi)\left(\frac{\partial\gamma_{xx}}{\partial x}+\frac{\partial \gamma_{xy}}{\partial y}\right)$$

$$\frac{\partial v}{\partial t}+u\frac{\partial v}{\partial x}+v\frac{\partial v}{\partial y} = -\frac{1}{\rho}\frac{\partial p}{\partial y}+\nu(1-P(\phi))\left(\frac{\partial^2 v}{\partial x^2}+\frac{\partial^2 v}{\partial y^2}\right) +GP(\phi)\left(\frac{\partial\gamma_{yx}}{\partial x}+\frac{\partial \gamma_{yy}}{\partial y}\right)$$

The interpolation polynomial $P(\phi)$ is choosen as:

$$P(\phi)=\frac{1.}{0.1215} \left(-\frac{\phi^3}{3}+0.55\phi^2-0.1\phi+\frac{0.29}{6}\right)$$

This particular choice of the polynomial has bad properties of discontinuty, so we also try standard polynomial:

$$ P(\phi) = \phi^3(10-15\phi+6\phi^2) $$

In [6]:
#PPhi = (phase < 1)*(1./0.1215*(-(phase**3)/3 + 0.55*phase**2 - 0.1*phase + 0.29/6.)-1)+1
#PPhi = (0.1 < phase) * PPhi
PPhi = phase**3 * (10. - 15*phase + 6*phase**2)
xVelocityEq = TransientTerm(var=xVelocity) +xVelocity*xVelocity.grad.dot([1,0])+yVelocity*xVelocity.grad.dot([0,1]) \
==  DiffusionTerm(coeff=viscosity*(1-PPhi), var=xVelocity) - pressure.grad.dot([1.,0.]) \
+G*PPhi*(gamma_xx.grad.dot([1,0])+gamma_xy.grad.dot([0,1]))
yVelocityEq = TransientTerm(var=yVelocity) +xVelocity*yVelocity.grad.dot([1,0])+yVelocity*yVelocity.grad.dot([0,1]) == \
 DiffusionTerm(coeff=viscosity*(1-PPhi), var=yVelocity) - pressure.grad.dot([0.,1.]) \
+G*PPhi*(gamma_xy.grad.dot([1,0])-gamma_xx.grad.dot([0,1]))

In [7]:
pressureEq = DiffusionTerm(var=pressure,coeff=1.) == 1./dt*(xVelocity.grad.dot([1,0])+yVelocity.grad.dot([0,1]))-((xVelocity.grad.dot([1,0]))**2 \
                                             +2*yVelocity.grad.dot([1,0])*xVelocity.grad.dot([0,1]) \
                                             +(yVelocity.grad.dot([0,1]))**2) 
 #                                               -G*PPhi*(gamma_xx.grad.dot([1,0]).grad.dot([1,0])- 
 #                                               gamma_xx.grad.dot([0,1]).grad.dot([0,1]) +2*gamma_xy.grad.dot([1,0]).grad.dot([0,1]))

Now we model thermal equation:

$$ \frac{\partial \Delta T}{\partial t} =  D_T \nabla^2\Delta T + 
 \frac{\partial\phi}{\partial t}+c(T-T_0)$$


In [8]:
DT = 2.25
q = Variable(0.)
T_0 = -0.1
heatEq = (TransientTerm(var=dT) == -PowerLawConvectionTerm(coeff=velocity, var=dT) + ImplicitSourceTerm(coeff=velocity.divergence,var=dT) +\
          DiffusionTerm(coeff=DT,var=dT) + TransientTerm(var=phase) \
          +q*T_0 - ImplicitSourceTerm(coeff=q, var=dT))

The governing equation for phase field which couples to thermal eq. is:

$$\tau_{\phi}\frac{D\phi}{D t}= \nabla D_{diff} \nabla \phi 
+\phi(1-\phi)m(\phi,\Delta T)-2s\phi|\nabla \theta|-\epsilon^2\phi|\nabla \theta|^2$$

where:

$$m(\phi,\Delta T) = \phi-\frac{1}{2}-\frac{\kappa_1}{\pi}arctan(\kappa_2 \Delta T)$$

represents source of anisotropy. $D_{diff}$ is diffusion tensor in two dimensions:

$$D_{diff} = \alpha^2(1+c\beta)*
\begin{pmatrix}
1+c\beta & -c\frac{\partial\beta}{\partial \psi}\\
c\frac{\partial\beta}{\partial \psi}&1+c\beta
\end{pmatrix}
$$

where $\beta=\frac{1-\Phi^2}{1+\Phi^2}, \Phi=tan(\frac{N}{2}\psi), \psi=\theta+
arctan\frac{\partial \phi/\partial y}{\partial \phi/\partial x}$,$\theta$ is the orientation, and N is the symmetry.

In [9]:
alpha=0.015
c=0.02
N=4.

psi = theta.arithmeticFaceValue + numerix.arctan2(phase.faceGrad[1],phase.faceGrad[0])

In [10]:
Phi = numerix.tan(N * psi/2)
PhiSq = Phi**2
beta = (1-PhiSq)/(1.+PhiSq)
DbetaDpsi = -N * 2 * Phi / (1+PhiSq)
Ddia = (1. + c * beta)

In [11]:
Doff = c * DbetaDpsi
I0 = Variable(value=((1,0),(0,1)))
I1 = Variable(value=((0,-1),(1,0)))
D = alpha**2 * Ddia * (Ddia * I0 + Doff * I1)

In [12]:
tau_phase = 3e-3
kappa1 = 0.9
kappa2 = 20.
epsilon = 0.008
s = 0.01
thetaMag = theta.grad.mag
phaseEq = (TransientTerm(coeff=tau_phase, var=phase) \
          == -tau_phase*(xVelocity*phase.grad.dot([1,0])+yVelocity*phase.grad.dot([0,1])) + DiffusionTerm(coeff=D,var=phase) + \
          ImplicitSourceTerm(coeff=((phase-0.5-kappa1/numerix.pi * numerix.arctan(kappa2 * dT)) \
                                   *(1-phase)-(2*s + epsilon**2 * thetaMag) * thetaMag),var=phase))

The governing equation for the crystalline orientation is given by:

$$ P(\epsilon|\nabla \theta|)\tau_{\theta}\phi^2\frac{D \theta}{Dt} =
\nabla \left[\phi^2\left(\frac{s}{|\nabla \theta|}+\epsilon^2\right)\nabla \theta\right]$$

where

$$P(\omega) = 1-exp(-\beta \omega)+\frac{\mu}{\epsilon}exp(-\beta \omega)$$

In [13]:
tau_theta = 3e-3
mu = 1e3
gamma = 1e3
thetaSmallValue = 1e-6
phaseMod = phase + ( phase < thetaSmallValue) * thetaSmallValue
beta_theta = 1e5
expo = epsilon * beta_theta * theta.grad.mag
expo = (expo < 100.) * (expo - 100.) + 100.
Pfunc = 1. + numerix.exp(-expo)*(mu / epsilon - 1.)

In [14]:
gradMagTheta = theta.faceGrad.mag
eps = 1. / gamma / 10.
gradMagTheta += (gradMagTheta < eps) * eps
IGamma = (gradMagTheta > 1. / gamma) * (1. /gradMagTheta - gamma) + gamma
v_theta = phase.arithmeticFaceValue * (s*IGamma + epsilon**2)
D_theta = phase.arithmeticFaceValue**2 * (s*IGamma + epsilon**2)

In [15]:
thetaEq = (TransientTerm(coeff=tau_theta * phaseMod**2*Pfunc, var=theta) == 
           -tau_theta * phaseMod**2 * Pfunc *(xVelocity*theta.grad.dot([1,0])+yVelocity*theta.grad.dot([0,1])) + \
          DiffusionTerm(coeff=D_theta, var=theta) + \
          PowerLawConvectionTerm(coeff=v_theta*(theta.faceGrad-theta.faceGradNoMod),var=theta))

In [16]:
x,y = mesh.cellCenters
#m = [[L/2.-dL*3,L/2.,0.2],[L/2.+dL*3,L/2.,0.3]]
m = [[L/2.,L/2.,0.2]]

for cx,cy,o in m:
    radius = dL * 3.
    seed = ((x-cx*ncells*dL)**2 + (y-cy*ncells*dL)**2) < radius**2
    phase[seed] = 1.
    theta[seed] = numerix.pi * (2. * o - 1.)
print 'seed:',seed,'radius:',radius,'N:',ncells,'dL:',dL

seed: [False False False ..., False False False] radius: 0.06 N: 50 dL: 0.02


In [17]:
vort = yVelocity.grad.dot([1,0])-xVelocity.grad.dot([0,1])
gamma_xxEq = (TransientTerm(var=gamma_xx) == -PowerLawConvectionTerm(coeff=velocity,var=gamma_xx)+ImplicitSourceTerm(coeff=velocity.divergence,var=gamma_xx) \
             +2. * xVelocity.grad.dot([1.,0]) - 2.*vort*gamma_xy)
gamma_xyEq = (TransientTerm(var=gamma_xy) == -PowerLawConvectionTerm(coeff=velocity,var=gamma_xy)+ImplicitSourceTerm(coeff=velocity.divergence,var=gamma_xy) \
             +xVelocity.grad.dot([0,1.])+yVelocity.grad.dot([1.,0]) + 2 * vort*gamma_xx)

In [18]:
dT.setValue(-0.5)

Set up the viewer for visualizing $p,u,v$ variables

In [19]:
if __name__=='__main__':
    viewer1 = MatplotlibVectorViewer(vars=velocity)
    viewer2 = Viewer(vars=(xVelocity), xmin=0.0, xmax=L,ymin=0.,ymax=L)
    viewer = Viewer(vars=(pressure,dT,gamma_xy,gamma_xx,phase,theta), xmin=0.0, xmax=L,ymin=0.,ymax=L)

In [20]:
eq = phaseEq & thetaEq & heatEq

In [21]:
total_time = 0.
duration = 2.
eps = 1e-7   
steps = 0
cfl = 0.

while total_time < duration:
    if total_time > 0.3:
        q.value = 100
        
    pres = 10e+10
    xVelocity.updateOld()
    yVelocity.updateOld()
    
    velocity[0] = xVelocity.arithmeticFaceValue
    velocity[1] = yVelocity.arithmeticFaceValue
    
    phase.updateOld()
    theta.updateOld()
    
    dT.updateOld()
    
    gamma_xy.updateOld()
    gamma_xx.updateOld()
    # do quasi-iteration of Poisson pressure equation
    while pres > eps:
        pres = pressureEq.sweep(var=pressure)
    #print 'pressure',pressure
    # solve momentum equations
    xres = xVelocityEq.solve(var=xVelocity,dt=dt)
    yres = yVelocityEq.solve(var=yVelocity,dt=dt)
    # next we solve phase equations with temperature
    #phaseEq.solve(var=phase,dt=dt)
    #thetaEq.solve(var=theta,dt=dt)
    #heatEq.solve(var=dT,dt=dt)
    velocity[0] = xVelocity.arithmeticFaceValue
    velocity[1] = yVelocity.arithmeticFaceValue
    
    eq.solve(dt=dt)
    gamma_xyEq.solve(var=gamma_xy,dt=dt)
    gamma_xxEq.solve(var=gamma_xx,dt=dt)
    
    
    #velocity[..., mesh.exteriorFaces.value] = 0.
    #velocity[0, mesh.facesTop.value] = U
    total_time += dt
    steps += 1
    #print 'step:',steps, 'x-res:', xres, 'y-res:',yres,'pres:',pres
    # Courant-Friedrich-Levy number 
    print 'cfl:', dt * (max(xVelocity)+max(yVelocity)) / dL
    if __name__ == '__main__':
        if steps % 10:
            viewer1.plot()
            viewer2.plot()
            viewer.plot()
        

cfl: 5.00000003238e-07
cfl: 1.00000000419e-06
cfl: 1.50000000454e-06
cfl: 2.00000000467e-06
cfl: 2.50000000544e-06
cfl: 3.00000000545e-06
cfl: 3.50000000589e-06
cfl: 4.00000000612e-06
cfl: 4.50000000614e-06
cfl: 5.00000000621e-06
cfl: 5.50000000598e-06
cfl: 6.00000000588e-06
cfl: 6.5000000057e-06
cfl: 7.00000000563e-06
cfl: 7.50000000581e-06
cfl: 8.000000006e-06
cfl: 8.50000000656e-06
cfl: 9.00000000711e-06
cfl: 9.50000000721e-06
cfl: 1.00000000073e-05
cfl: 1.05000000074e-05
cfl: 1.10000000079e-05
cfl: 1.15000000083e-05
cfl: 1.20000000084e-05
cfl: 1.25000000084e-05
cfl: 1.30000000086e-05
cfl: 1.35000000088e-05
cfl: 1.40000000091e-05
cfl: 1.45000000095e-05
cfl: 1.500000001e-05
cfl: 1.55000000101e-05
cfl: 1.60000000102e-05
cfl: 1.65000000105e-05
cfl: 1.70000000108e-05
cfl: 1.75000000114e-05
cfl: 1.80000000114e-05
cfl: 1.85000000117e-05
cfl: 1.9000000012e-05
cfl: 1.95000000125e-05
cfl: 2.00000000125e-05
cfl: 2.05000000128e-05
cfl: 2.10000000131e-05
cfl: 2.15000000135e-05
cfl: 2.2000000013

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/fipy/variables/variable.py:1165: RuntimeWarning: invalid value encountered in divide
  return self._BinaryOperatorVariable(lambda a,b: a/b, other)
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/fipy/terms/powerLawConvectionTerm.py:100: RuntimeWarning: invalid value encountered in less
  P = numerix.where(abs(P) < eps, eps, P)
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/fipy/terms/powerLawConvectionTerm.py:102: RuntimeWarning: invalid value encountered in greater
  alpha = numerix.where(                  P > 10.,                     (P - 1.) / P,   0.5)
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/fipy/terms/powerLawConvectionTerm.py:106: RuntimeWarning: invalid value encountered in less_equal
  alpha = numerix.where(  (10. >= P) & (P > eps), ((P-1.) + tmpSqr*tmpSqr*tmp) / P